# Graph-to-Combinatorial GraphCurve Lifting Tutorial

***
This notebook shows how to import a dataset, with the desired lifting, and how to run a neural network using the loaded data.

The notebook is divided into sections:

- [Loading the dataset](#loading-the-dataset) loads the config files for the data and the desired tranformation, createsa a dataset object and visualizes it.
- [Loading and applying the lifting](#loading-and-applying-the-lifting) defines a simple neural network to test that the lifting creates the expected incidence matrices.
- [Create and run a simplicial nn model](#create-and-run-a-simplicial-nn-model) simply runs a forward pass of the model to check that everything is working as expected.

***
***

Note that for simplicity the notebook is setup to use a simple graph. However, there is a set of available datasets that you can play with.

To switch to one of the available datasets, simply change the *dataset_name* variable in [Dataset config](#dataset-config) to one of the following names:

* cocitation_cora
* cocitation_citeseer
* cocitation_pubmed
* MUTAG
* NCI1
* NCI109
* PROTEINS_TU
* AQSOL
* ZINC
***

### Imports and utilities

In [ ]:
# With this cell any imported module is reloaded before each cell execution
%load_ext autoreload
%autoreload 2
from modules.data.load.loaders import GraphLoader
from modules.data.preprocess.preprocessor import PreProcessor
from modules.utils.utils import (
    describe_data,
    load_dataset_config,
    load_model_config,
    load_transform_config,
)

## Graphic Curve Lifting Explanation

In order to make a Graph Curve Matroid, we need the following (from some graph $G = (V, E)$):
- The graphic matroid of the graph $G$: $M(G) = (E, \mathcal{I})$
- Its dual $M^* = M^*(G) = (E, \mathcal{I}^*)$, the bond matroid, and its rank function: $\mathit{rank}^*: 2^E \to \mathbb{Z}_+$.


## Dual Matroid
A dual matroid of another matroid is where the bases of the dual matroid are the complements of the original matroid's bases.

## Graphic Matroid
A graphic matroid is a matroid where its bases are the edges that make the maximum spanning trees of the graph $G$.

## Bond Matroid
A bond matroid is a matroid where its independent sets are the edges that when removed, retain the number of connected components of the graph. One thing is important to note about this matroid: from graph theory, we know that unique duals of nonplanar graphs do not exist, meaning it is harder to analyze the graphs by their (nonunique) duals. However, for a graphic matroid, a unique dual **always** exists, which is a nice property. 

## Graphic Curve Matroid
The idea now is to use the bond matroids, but there is one problem: the ground set is based on edges, instead of vertices. We can always make a trivial reduction from edges to vertices by means of incidence, however, some natural reductions exist. One of these is called the Graph Curve Matroids, introduced recently by Alheydis Geiger, Kevin Kühn, and Raluca Vlad.

For the Graph Curve Matroids $M_g = (V, \mathcal{I})$, if $A \subseteq V$:
- If $A$ is a cycle, then $A$ is dependent in $M_g$.
- If $A$ is acyclic, it is dependent if its removal connects G. It is independent if its removal retains the connectedness.

The above was shown in [1].

Moreover, $A \subseteq V$ is a circuit in $M_g$ if $A$ is a cycle or $A$ is acyclic and its disconnection increases the number of connected components by $1$. In the example of the lifting below, we can see the circuits of $M_g$ are precisely the 3 clusters of nodes.

[1] Geiger, Alheydis, Kevin Kuehn, and Raluca Vlad. "Graph Curve Matroids." arXiv preprint arXiv:2311.08332 (2023). https://arxiv.org/abs/2311.08332.

In [ ]:
# Define transformation type and id
transform_type = "liftings"
# If the transform is a topological lifting, it should include both the type of the lifting and the identifier
transform_id = "graph2combinatorial/curve_lifting"

# Read yaml file
transform_config = {
    "lifting": load_transform_config(transform_type, transform_id)
    # other transforms (e.g. data manipulations, feature liftings) can be added here
}

## Loading the Dataset: Manual

In [ ]:
dataset_name = "manual_dataset"
dataset_config = load_dataset_config(dataset_name)
loader = GraphLoader(dataset_config)

In [ ]:
dataset = loader.load()
describe_data(dataset)

We than apply the transform via our `PreProcesor`:

In [ ]:
lifted_dataset = PreProcessor(dataset, transform_config, loader.data_dir)
describe_data(lifted_dataset)

By the authors' own admission, graphs that are not trivalent ($\forall v \in V, \mathit{deg}(v) >= 3$) are not interesting. [1]
Therefore, using trivalent, 2-connected graphs are better.

One limitation is that the graphic curve matroid is hard to compute, it requires checking every subset of a possible candidate set to make the graphic curve matroid. Not to mention that creating the graphic and cographic matroids are also expensive as well.

## Create and Run a Combinatorial NN Model

In this section a simple model is created to test that the used lifting works as intended. In this case the model uses the `adjacency_0`, `adjacency_1`, `adjacency_2`, `incidence_1`, `incidence_2` so the lifting should make sure to add them to the data.

In [ ]:
from modules.models.combinatorial.hmc import HMCModel

model_type = "combinatorial"
model_id = "hmc"
model_config = load_model_config(model_type, model_id)

model = HMCModel(model_config, dataset_config)

In [ ]:
y_hat = model(lifted_dataset.get(0))

If everything is correct the cell above should execute without errors. 